In [1]:
#1234

In [2]:
!pip install opendatasets
import opendatasets as od
od.download("https://www.kaggle.com/datasets/mohammedalrajeh/quran-recitations-for-audio-classification")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: rehman047
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/mohammedalrajeh/quran-recitations-for-audio-classification


100%|██████████| 2.12G/2.12G [00:27<00:00, 83.8MB/s]


In [26]:
import torch
from torch import nn
from torch.optim import SGD
import librosa
from torch.utils.data import Dataset,DataLoader
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from skimage.transform import resize

device="cuda" if torch.cuda.is_available else 'cpu'

In [30]:
# /content/quran-recitations-for-audio-classification/Dataset/Dataset/AbdulBari_Althubaity/abdulbari_001.wav

In [31]:
df=pd.read_csv("/content/quran-recitations-for-audio-classification/files_paths.csv")

In [34]:
df.head().iloc[0,0]

'/content/quran-recitations-for-audio-classification/Dataset/Dataset/Mohammed_Aluhaidan/lohaidan_171.wav'

In [33]:
df['FilePath']='/content/quran-recitations-for-audio-classification/Dataset'+df['FilePath'].str[1:]

In [38]:
df.head()

,FilePath,Class
0,/content/quran-recitations-for-audio-classific...,Mohammed_Aluhaidan
1,/content/quran-recitations-for-audio-classific...,Mohammed_Aluhaidan
2,/content/quran-recitations-for-audio-classific...,Mohammed_Aluhaidan
3,/content/quran-recitations-for-audio-classific...,Mohammed_Aluhaidan
4,/content/quran-recitations-for-audio-classific...,Mohammed_Aluhaidan


In [40]:
len(df['Class'].unique())

12

In [41]:
df['Class'].unique()

array(['Mohammed_Aluhaidan', 'Yasser_Aldossary', 'Maher_Almuaiqly',
       'Nasser_Alqutami', 'AbdulBari_Althubaity', 'Bander_Balilah',
       'Ali_Alhothaify', 'Saud_Alshuraim', 'Mohammed_Ayoub',
       'AbdulRahman_Alsudais', 'Saad_Alghamdi', 'Abdullah_Albuaijan'],
      dtype=object)

In [42]:
df.shape

(6687, 2)

In [43]:
df['Class'].value_counts()

,count
Class,
Saud_Alshuraim,696
Saad_Alghamdi,688
AbdulRahman_Alsudais,648
Yasser_Aldossary,576
Ali_Alhothaify,576
Bander_Balilah,576
Maher_Almuaiqly,576
Abdullah_Albuaijan,504
Mohammed_Ayoub,480
